# Tarea 1: DS Lab
- Ricardo Arancibia
- José Díaz
- Javier Santibáñez

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## P1

## P2

In [ ]:
def sinplot(flip=1):
    x = np.linspace(0, 14, 100)
    for i in range(1, 7):
        plt.plot(x, np.sin(x + i * .5) * (7 - i) * flip)
sinplot()

In [ ]:
def estilo(sty = "whitegrid"):
    param = {'axes.facecolor': 'white',
             'axes.edgecolor': 'black',
             'axes.grid': True,
             'axes.axisbelow': 'line',
             'axes.labelcolor': 'black',
             'figure.facecolor': (1, 1, 1, 0),
             'grid.color': '#b0b0b0',
             'grid.linestyle': '-',
             'text.color': 'black',
             'xtick.color': 'red',
             'ytick.color': 'black',
             'xtick.direction': 'out',
             'ytick.direction': 'out',
             'lines.solid_capstyle': 'projecting',
             'patch.edgecolor': 'black',
             'image.cmap': 'viridis',
             'font.family': ['sans-serif'],
             'font.sans-serif': ['DejaVu Sans',
             'Bitstream Vera Sans',
             'Computer Modern Sans Serif',
             'Lucida Grande',
             'Verdana',
             'Geneva',
             'Lucid',
             'Arial',
             'Helvetica',
             'Avant Garde',
             'sans-serif'],
             'patch.force_edgecolor': False,
             'xtick.bottom': True,
             'xtick.top': False,
             'ytick.left': True,
             'ytick.right': False,
             'axes.spines.left': True,
             'axes.spines.bottom': True,
             'axes.spines.right': False,
             'axes.spines.top': False}
    sns.set_style(sty, param)

In [ ]:
estilo()
sinplot()

## P3

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin

In [ ]:
# 1
class RegresionBayesianaEmpirica(BaseEstimator, RegressorMixin):

    def __init__(self, alpha_0, beta_0, tol=1e-5, maxiter=200):
        self.set_params(alpha_0=alpha_0,beta_0=beta_0,tol=tol,maxiter=maxiter)

    def get_posteriori(self, X, y, alpha, beta):
        N,d = X.shape
        S_N_1 = alpha*np.identity(d) + beta*(X.T @ X)
        S_N = np.linalg.inv(S_N_1)
        m_N = beta*((S_N @ X.T) @ y)
        return m_N, S_N

    def fit(self, X, y):
        N,d = X.shape
        alpha_old = self.alpha_0
        beta_old = self.beta_0
        print('Iteracion: 1')
        print('valores actuales, alpha:{}, beta:{}'.format(alpha_old,beta_old))
        lambdas, vectores = np.linalg.eig(beta_old*(X.T @ X))
        lambdas = np.real_if_close(lambdas,tol=1)
        gamma = np.sum(lambdas/(alpha_old + lambdas))
        m_N, S_N = self.get_posteriori(X,y,alpha_old,beta_old)
        alpha_new = gamma/(m_N.T @ m_N)
        beta_new = (N-gamma)/(np.sum((y-m_N.T@X.T)**2))
        for k in range(2, self.maxiter+1): 
            err_alpha = abs(alpha_old - alpha_new)
            err_beta = abs(beta_old - beta_new)
            if (err_alpha < self.tol) and (err_beta < self.tol):
                break
            else:
                print('Iteracion: ', k)
                print('valores actuales, alpha:{}, beta:{}'.format(alpha_new,beta_new))
                alpha_old = alpha_new
                beta_old = beta_new
                lambdas, vectores = np.linalg.eig(beta_old*(X.T @ X))
                lambdas = np.real_if_close(lambdas,tol=1)
                gamma = np.sum(lambdas/(alpha_old + lambdas))
                m_N, S_N = self.get_posteriori(X,y,alpha_old,beta_old)
                alpha_new = gamma/(m_N.T @ m_N)
                beta_new = (N-gamma)/(np.sum((y-m_N.T@X.T)**2))
        self.alpha_0 = alpha_new
        self.beta_0 = beta_new
        print('\nvalores finales, alpha:{}, beta:{}'.format(alpha_new,beta_new))
        m_N, S_N = self.get_posteriori(X,y,alpha_new,beta_new)
        self.m_N = m_N
        self.S_N = S_N

    def predict(self, X_, return_std=False):
        y_ = X_ @ self.m_N
        y_std = np.sqrt(1/self.beta_0 + np.diag((X_ @ self.S_N) @ X_.T))
        if return_std:
            return y_, y_std
        else:
            return y_    

In [ ]:
# ejemplito del fit
mat = np.array([[1, 0, 0], [0, 1, 0], [0,0,1], [0,0,0]])
mat_y = np.array([10, 12,7,5])
rbe = RegresionBayesianaEmpirica(0.5, 2/3)
rbe.fit(mat, mat_y)
print('prediccion: ',rbe.predict(np.array([[1,2,3]]))) 

In [ ]:
# 2
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, PolynomialFeatures
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
data = pd.read_csv('data.csv')
data.columns

In [106]:
data['n_garajes'].value_counts()

1    6186
2    5070
0    1682
3     692
4     483
5       5
Name: n_garajes, dtype: int64

In [107]:
## JS: Hay que definir las columnas numericas y categoricas
# data.columns
cat_cols = ['tipo_producto', 'rent_type', 'location','furnished','Cluster']

num_perc_cols = ['n_rooms', 'n_bath','n_garajes', 'metrocuadrado_index',
       'trabajoinf_ninos_5_17_anos_perc',
       'trabajoinfampliado_ninos_5_17_anos_perc',
       'jovenes_14_24_anos_nini_perc',
       'indice_envegecimiento',
       'jefe_mujer_perc',
       'adultos_mayores_pobres_perc',
       'indice_inseguridad', 
       'areas_verdes_perc']

num_cols =['Densidad']

df_X = data[cat_cols + num_perc_cols + num_cols]
print(df_X.shape)
df_y = data['price_by_m2']

(14118, 18)


In [108]:
# categorical pipeline
cat_pipe = Pipeline([('encoding', OneHotEncoder(handle_unknown='ignore'))])

# numerical pipeline
num_perc_pipe = Pipeline([('scaler', MinMaxScaler()), ('poly', PolynomialFeatures(degree=3))])
num_pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures(degree=3))])

prep = ColumnTransformer(transformers=[('num_perc', num_perc_pipe, num_perc_cols),('num', num_pipe, num_cols), ('cat', cat_pipe, cat_cols)])

In [109]:
# 3
from sklearn.model_selection import train_test_split

def pipe_run(X, y, pipeline):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    RMSE = np.sqrt(np.mean((y_pred - y_test)**2))
    R2 = pipeline.score(X_test, y_test)
    print('RMSE: ', RMSE)
    print('R^2: ', R2)
    return pipeline, RMSE, R2

In [ ]:
# Probar varios alpha y beta
alphas = np.linspace(1e-10, 1e-5, 3, endpoint=True)
betas = np.linspace(1e-10, 1e-5, 3, endpoint=True)
params = []
R2s = []

for alpha in [1e-9]:#alphas:
    for beta in [1e-9]:#betas:
        el_pipe = Pipeline([('prep', prep), ('regressor', RegresionBayesianaEmpirica(alpha,beta,tol=1e-15))])
        regressor, RMSE, R2 = pipe_run(df_X, df_y, el_pipe)
        params.append((alpha,beta))
        R2s.append(R2)

In [ ]:
alpha_opt, beta_opt = params[R2s.index(max(R2s))]
print('Modelo Optimo')
print('alpha:{}, beta:{}'.format(alpha_opt, beta_opt))
el_pipe = Pipeline([('prep', prep), ('regressor', RegresionBayesianaEmpirica(alpha_opt,beta_opt))])
regressor, RMSE, R2 = pipe_run(df_X, df_y, el_pipe)

In [ ]:
# 4

## JS: Hay que REdefinir las columnas numericas y categoricas... obtenidas de EDA
cat_cols = ['tipo_producto', 'Cluster', 'rent_type', 'location','furnished']

num_perc_cols = ['n_rooms', 'n_bath','n_garajes', 'metrocuadrado_index',
       'trabajoinf_ninos_5_17_anos_perc',
       'trabajoinfampliado_ninos_5_17_anos_perc',
       'jovenes_14_24_anos_nini_perc',
       'indice_envegecimiento',
       'jefe_mujer_perc',
       'adultos_mayores_pobres_perc',
       'indice_inseguridad', 
       'areas_verdes_perc']

num_cols =['Densidad']

df_X = data[cat_cols + num_perc_cols + num_cols]
# print(df_X.shape)
df_y = data['price_by_m2']

prep = ColumnTransformer(transformers=[('num_perc', num_perc_pipe, num_perc_cols),('num', num_pipe, num_cols), ('cat', cat_pipe, cat_cols)])

el_pipe = Pipeline([('prep', prep), ('regressor', RegresionBayesianaEmpirica(alpha_opt,beta_opt))])
regressor, RMSE, R2 = pipe_run(df_X, df_y, el_pipe)

In [ ]:
import pickle
# guardar modelo
filename = 'modelo.pkl'
pickle.dump(regressor, open(filename, 'wb'))
 
# para cargar modelo:
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)

In [ ]:
# 5
from sklearn.linear_model import BayesianRidge

ridge_pipe = Pipeline([('prep', prep), ('regressor_ridge', BayesianRidge())])
regressor_ridge, RMSE_ridge, R2_ridge = pipe_run(df_X, df_y, ridge_pipe)